# Multi-Agent Pricing and Rebalancing Visualization

This notebook demonstrates various ways to visualize:
1. **Price scalars per region** for each agent
2. **Rebalancing flows** (vehicle movements) for each agent
3. **Net differences** between agents (price competition, flow imbalances)
4. **Temporal evolution** of pricing and rebalancing over time

We'll use Kepler.gl for interactive map visualizations.

In [22]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
from keplergl import KeplerGl
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

# For creating flow lines
from shapely.geometry import LineString, Point

## 1. Load Geographic Data

**Note:** We filter for Manhattan zones only (districtcode starting with '1').

In [23]:
# Load NYC zone geometries
zones_gdf = gpd.read_file('data/nyc_zones.geojson')
print(f"Loaded {len(zones_gdf)} total zones")

# Filter for Manhattan zones only (districtcode starting with '1')
zones_gdf = zones_gdf[zones_gdf['districtcode'].astype(str).str.startswith('1')].copy()
zones_gdf = zones_gdf.reset_index(drop=True)

print(f"Filtered to {len(zones_gdf)} Manhattan zones")
print(f"Columns: {zones_gdf.columns.tolist()}")
zones_gdf.head()

Loaded 59 total zones
Filtered to 12 Manhattan zones
Columns: ['districtcode', 'shape_area', 'objectid', 'shape_length', 'district', 'geometry']


,districtcode,shape_area,objectid,shape_length,district,geometry
0,111,66146445.0668882,36,59502.7831201084,MN11,"MULTIPOLYGON (((-73.93381 40.81651, -73.93383 ..."
1,101,40710976.0632209,72,62155.4849343709,MN01,"MULTIPOLYGON (((-74.01114 40.72581, -74.01102 ..."
2,110,39080798.8082581,57,35834.0231815627,MN10,"MULTIPOLYGON (((-73.93445 40.83598, -73.93456 ..."
3,102,37719022.1000202,17,34894.0812518556,MN02,"MULTIPOLYGON (((-73.99684 40.73736, -73.99362 ..."
4,103,46882269.9811591,18,30452.376332722,MN03,"MULTIPOLYGON (((-73.98878 40.73397, -73.98718 ..."


## 2. Load Trained Model Data

Load the visualization data saved from running the model in test mode. 

The pickle file contains:
- **agent_price_scalars**: Price scalars per region per timestep `[timesteps, regions]`
- **agent_reb_actions**: Rebalancing action probabilities per region per timestep `[timesteps, regions]`
- **agent_acc_temporal**: Vehicle availability per region per timestep `[timesteps, regions]`
- **metadata**: Information about the simulation (mode, city, num_regions, checkpoint_path, etc.)

**To generate this data file:**
Run your model in test mode: `python main_a2c_multi_agent.py --test --checkpoint_path <your_model> --city nyc_manhattan --mode 2`

The file will be saved as: `saved_files/visualization_data/<checkpoint_path>_viz_data.pkl`

In [24]:
import pickle
import os

# Path to visualization data (update this to your checkpoint name)
# Format: saved_files/visualization_data/<checkpoint_path>_viz_data.pkl
checkpoint_name = 'test_viz_checkpoint'  # UPDATE THIS to your model name
viz_data_path = f'saved_files/visualization_data/{checkpoint_name}_viz_data.pkl'

# Check if file exists, if not, use dummy data
if os.path.exists(viz_data_path):
    print(f"Loading visualization data from: {viz_data_path}")
    with open(viz_data_path, 'rb') as f:
        viz_data = pickle.load(f)
    
    # Extract data
    agents = [0, 1]
    num_regions = viz_data['metadata']['num_regions']
    num_timesteps = viz_data['metadata']['num_timesteps']
    city = viz_data['metadata']['city']
    mode = viz_data['metadata']['mode']
    checkpoint_path = viz_data['metadata'].get('checkpoint_path', 'unknown')
    
    print(f"\nLoaded data from {city} (mode {mode}):")
    print(f"  - Checkpoint: {checkpoint_path}")
    print(f"  - Number of regions: {num_regions}")
    print(f"  - Number of timesteps: {num_timesteps}")
    print(f"  - Agents: {agents}")
    
    # Extract agent actions
    agent_price_scalars = viz_data['agent_price_scalars']  # {agent_id: [timesteps, regions]}
    agent_reb_actions = viz_data['agent_reb_actions']      # {agent_id: [timesteps, regions]}
    agent_acc = viz_data['agent_acc_temporal']             # {agent_id: [timesteps, regions]}
    
    # Convert to the format expected by visualization code
    # For prices: we need agent_price[agent_id][(i,j)][t] = price_scalar
    agent_price = {}
    for agent_id in agents:
        agent_price[agent_id] = {}
        if len(agent_price_scalars[agent_id]) > 0:
            for i in range(num_regions):
                for j in range(num_regions):
                    if i != j:
                        agent_price[agent_id][(i, j)] = {}
                        for t in range(len(agent_price_scalars[agent_id])):
                            # Use the price scalar for origin region i
                            agent_price[agent_id][(i, j)][t] = agent_price_scalars[agent_id][t][i]
    
    # For rebalancing: we already have the desired distribution, 
    # we can create dummy flows for visualization based on the difference
    agent_rebFlow = {}
    for agent_id in agents:
        agent_rebFlow[agent_id] = {}
        if len(agent_reb_actions[agent_id]) > 0:
            for i in range(num_regions):
                for j in range(num_regions):
                    agent_rebFlow[agent_id][(i, j)] = {}
                    for t in range(len(agent_reb_actions[agent_id])):
                        # Simple heuristic: flow from i to j based on desired distribution difference
                        if i != j and t > 0:
                            # Flow proportional to difference in desired distribution
                            flow = agent_reb_actions[agent_id][t][j] - agent_reb_actions[agent_id][t-1][j]
                            agent_rebFlow[agent_id][(i, j)][t] = max(0, flow * 100)  # Scale up for visibility
                        else:
                            agent_rebFlow[agent_id][(i, j)][t] = 0
        else:
            # No rebalancing data available (e.g., mode 1 or 3)
            for i in range(num_regions):
                for j in range(num_regions):
                    agent_rebFlow[agent_id][(i, j)] = {}
                    for t in range(num_timesteps):
                        agent_rebFlow[agent_id][(i, j)][t] = 0
    
    # Convert agent_acc to expected format: agent_acc[agent_id][region][t]
    agent_acc_formatted = {}
    for agent_id in agents:
        agent_acc_formatted[agent_id] = {}
        if len(agent_acc[agent_id]) > 0:
            for region in range(num_regions):
                agent_acc_formatted[agent_id][region] = {}
                for t in range(len(agent_acc[agent_id])):
                    agent_acc_formatted[agent_id][region][t] = agent_acc[agent_id][t][region]
    agent_acc = agent_acc_formatted
    
    print(f"\n✓ Data successfully loaded and formatted")
    print(f"  - Price data available: {all(len(agent_price_scalars[a]) > 0 for a in agents)}")
    print(f"  - Rebalancing data available: {all(len(agent_reb_actions[a]) > 0 for a in agents)}")
    print(f"  - Availability data available: {all(len(agent_acc[a]) > 0 for a in agents)}")
    
else:
    print(f"⚠ Visualization data file not found: {viz_data_path}")
    print("Generating dummy data instead...")
    print("To use real data, run your model in test mode first:")
    print("  python main_a2c_multi_agent.py --test --checkpoint_path <model_name> --city nyc_manhattan --mode 2")
    
    # Generate dummy data (same as before)
    num_regions = len(zones_gdf)
    num_timesteps = 20
    agents = [0, 1]
    
    agent_price = {}
    for agent_id in agents:
        agent_price[agent_id] = {}
        for i in range(num_regions):
            for j in range(num_regions):
                if i != j:
                    agent_price[agent_id][(i, j)] = {}
                    for t in range(num_timesteps):
                        if agent_id == 0:
                            agent_price[agent_id][(i, j)][t] = np.random.uniform(0.7, 1.1)
                        else:
                            agent_price[agent_id][(i, j)][t] = np.random.uniform(0.9, 1.3)
    
    agent_rebFlow = {}
    for agent_id in agents:
        agent_rebFlow[agent_id] = {}
        for i in range(num_regions):
            for j in range(num_regions):
                agent_rebFlow[agent_id][(i, j)] = {}
                for t in range(num_timesteps):
                    if np.random.random() < 0.15:
                        agent_rebFlow[agent_id][(i, j)][t] = np.random.randint(1, 10)
                    else:
                        agent_rebFlow[agent_id][(i, j)][t] = 0
    
    agent_acc = {}
    for agent_id in agents:
        agent_acc[agent_id] = {}
        for region in range(num_regions):
            agent_acc[agent_id][region] = {}
            for t in range(num_timesteps):
                base = 20 if agent_id == 0 else 25
                agent_acc[agent_id][region][t] = max(0, base + np.random.randint(-10, 15))
    
    print("Generated dummy data")

print(f"\nData ready for visualization!")

Loading visualization data from: saved_files/visualization_data/test_viz_checkpoint_viz_data.pkl

Loaded data from nyc_manhattan (mode 3):
  - Checkpoint: test_viz_checkpoint
  - Number of regions: 12
  - Number of timesteps: 5
  - Agents: [0, 1]

✓ Data successfully loaded and formatted
  - Price data available: True
  - Rebalancing data available: False
  - Availability data available: True

Data ready for visualization!


## 3. Visualization 1: Regional Price Heatmap (Per Agent)

Show average pricing scalar aggregated by origin region for each agent.

In [4]:
def create_price_heatmap_data(agent_price, zones_gdf, timestep=None):
    """
    Create GeoDataFrame for price heatmap visualization.
    If timestep is None, use average across all timesteps.
    """
    price_data = []
    
    for agent_id in [0, 1]:
        region_prices = {}
        
        # Calculate average price for each origin region
        for (i, j), time_prices in agent_price[agent_id].items():
            if timestep is not None:
                if timestep in time_prices:
                    price = time_prices[timestep]
                else:
                    continue
            else:
                price = np.mean(list(time_prices.values()))
            
            if i not in region_prices:
                region_prices[i] = []
            region_prices[i].append(price)
        
        # Average prices for each origin region
        for region_id, prices in region_prices.items():
            if region_id < len(zones_gdf):
                price_data.append({
                    'region_id': region_id,
                    'agent_id': agent_id,
                    'agent_name': f'Agent {agent_id}',
                    'avg_price': np.mean(prices),
                    'min_price': np.min(prices),
                    'max_price': np.max(prices),
                    'geometry': zones_gdf.iloc[region_id].geometry
                })
    
    return gpd.GeoDataFrame(price_data, crs=zones_gdf.crs)

# Create price heatmap data
price_gdf = create_price_heatmap_data(agent_price, zones_gdf)
print(f"Created price heatmap data: {len(price_gdf)} records")
price_gdf.head()

Created price heatmap data: 24 records


,region_id,agent_id,agent_name,avg_price,min_price,max_price,geometry
0,0,0,Agent 0,0.905133,0.877949,0.947924,"MULTIPOLYGON (((-73.93381 40.81651, -73.93383 ..."
1,1,0,Agent 0,0.903409,0.831103,0.939454,"MULTIPOLYGON (((-74.01114 40.72581, -74.01102 ..."
2,2,0,Agent 0,0.884187,0.854543,0.908453,"MULTIPOLYGON (((-73.93445 40.83598, -73.93456 ..."
3,3,0,Agent 0,0.883383,0.855405,0.908707,"MULTIPOLYGON (((-73.99684 40.73736, -73.99362 ..."
4,4,0,Agent 0,0.905418,0.860989,0.952285,"MULTIPOLYGON (((-73.98878 40.73397, -73.98718 ..."


In [6]:
# Visualize price heatmap with Kepler.gl
map_price = KeplerGl(height=600)
map_price.add_data(data=price_gdf, name='Agent Pricing')

# Configuration for price heatmap
price_config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': 40.75,
            'longitude': -73.95,
            'zoom': 10
        },
        'visState': {
            'filters': [
                {
                    'dataId': ['Agent Pricing'],
                    'id': 'agent_filter',
                    'name': ['agent_name'],
                    'type': 'multiSelect',
                    'value': ['Agent 0', 'Agent 1'],
                    'enlarged': True
                }
            ],
            'layers': [
                {
                    'type': 'geojson',
                    'config': {
                        'dataId': 'Agent Pricing',
                        'label': 'Price Heatmap',
                        'color': [18, 147, 154],
                        'columns': {'geojson': 'geometry'},
                        'isVisible': True,
                        'visConfig': {
                            'opacity': 0.8,
                            'strokeOpacity': 0.8,
                            'thickness': 0.5,
                            'strokeColor': [255, 255, 255],
                            'colorRange': {
                                'name': 'Global Warming',
                                'type': 'sequential',
                                'category': 'Uber',
                                'colors': ['#5A1846', '#900C3F', '#C70039', '#E3611C', '#F1920E', '#FFC300']
                            },
                            'strokeColorRange': {
                                'name': 'Global Warming',
                                'type': 'sequential',
                                'category': 'Uber',
                                'colors': ['#5A1846', '#900C3F', '#C70039', '#E3611C', '#F1920E', '#FFC300']
                            },
                            'radius': 10,
                            'filled': True,
                            'stroked': True,
                            'heightRange': [0, 500],
                            'elevationScale': 5,
                            'enableElevationZoomFactor': True,
                            'fixedHeight': False,
                            'wireframe': False
                        },
                        'colorField': {'name': 'avg_price', 'type': 'real'},
                        'colorScale': 'quantile'
                    }
                }
            ]
        }
    }
}

map_price.config = price_config
map_price

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'mapState': {'latitude': 40.75, 'longitude': -73.95, 'zoom': 10},…

## 4. Visualization 2: Price Difference Heatmap (Competition)

Show the difference between Agent 0 and Agent 1 pricing to visualize competitive dynamics.

In [7]:
def create_price_difference_data(agent_price, zones_gdf):
    """
    Calculate price differences (Agent 0 - Agent 1) by region.
    Negative values mean Agent 0 prices lower, positive means Agent 0 prices higher.
    """
    price_diff_data = []
    
    region_prices = {0: {}, 1: {}}
    
    # Calculate average prices per region for each agent
    for agent_id in [0, 1]:
        for (i, j), time_prices in agent_price[agent_id].items():
            avg_price = np.mean(list(time_prices.values()))
            if i not in region_prices[agent_id]:
                region_prices[agent_id][i] = []
            region_prices[agent_id][i].append(avg_price)
    
    # Calculate differences
    all_regions = set(region_prices[0].keys()) | set(region_prices[1].keys())
    for region_id in all_regions:
        if region_id < len(zones_gdf):
            price_0 = np.mean(region_prices[0].get(region_id, [1.0]))
            price_1 = np.mean(region_prices[1].get(region_id, [1.0]))
            diff = price_0 - price_1
            
            price_diff_data.append({
                'region_id': region_id,
                'price_agent0': price_0,
                'price_agent1': price_1,
                'price_diff': diff,
                'price_diff_pct': (diff / price_1 * 100) if price_1 > 0 else 0,
                'competitive_advantage': 'Agent 0 Lower' if diff < 0 else 'Agent 1 Lower' if diff > 0 else 'Equal',
                'geometry': zones_gdf.iloc[region_id].geometry
            })
    
    return gpd.GeoDataFrame(price_diff_data, crs=zones_gdf.crs)

price_diff_gdf = create_price_difference_data(agent_price, zones_gdf)
print(f"Price difference range: {price_diff_gdf['price_diff'].min():.3f} to {price_diff_gdf['price_diff'].max():.3f}")
price_diff_gdf.head()

Price difference range: -0.221 to -0.182


,region_id,price_agent0,price_agent1,price_diff,price_diff_pct,competitive_advantage,geometry
0,0,0.905133,1.087020,-0.181887,-16.732631,Agent 0 Lower,"MULTIPOLYGON (((-73.93381 40.81651, -73.93383 ..."
1,1,0.903409,1.086713,-0.183304,-16.867737,Agent 0 Lower,"MULTIPOLYGON (((-74.01114 40.72581, -74.01102 ..."
2,2,0.884187,1.101823,-0.217636,-19.752375,Agent 0 Lower,"MULTIPOLYGON (((-73.93445 40.83598, -73.93456 ..."
3,3,0.883383,1.104509,-0.221126,-20.020300,Agent 0 Lower,"MULTIPOLYGON (((-73.99684 40.73736, -73.99362 ..."
4,4,0.905418,1.095085,-0.189667,-17.319851,Agent 0 Lower,"MULTIPOLYGON (((-73.98878 40.73397, -73.98718 ..."


In [8]:
# Visualize price differences
map_price_diff = KeplerGl(height=600)
map_price_diff.add_data(data=price_diff_gdf, name='Price Competition')

# Use diverging color scheme (red = Agent 0 lower, blue = Agent 1 lower)
diff_config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': 40.75,
            'longitude': -73.95,
            'zoom': 10
        },
        'visState': {
            'layers': [
                {
                    'type': 'geojson',
                    'config': {
                        'dataId': 'Price Competition',
                        'label': 'Price Difference (Agent 0 - Agent 1)',
                        'color': [18, 147, 154],
                        'columns': {'geojson': 'geometry'},
                        'isVisible': True,
                        'visConfig': {
                            'opacity': 0.8,
                            'strokeOpacity': 0.8,
                            'thickness': 0.5,
                            'strokeColor': [255, 255, 255],
                            'colorRange': {
                                'name': 'Diverging RdBu',
                                'type': 'diverging',
                                'category': 'ColorBrewer',
                                'colors': ['#2166AC', '#4393C3', '#92C5DE', '#D1E5F0', '#F7F7F7', '#FDDBC7', '#F4A582', '#D6604D', '#B2182B']
                            },
                            'filled': True,
                            'stroked': True,
                            'wireframe': False
                        },
                        'colorField': {'name': 'price_diff', 'type': 'real'},
                        'colorScale': 'quantize'
                    }
                }
            ]
        }
    }
}

map_price_diff.config = diff_config
map_price_diff

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'mapState': {'latitude': 40.75, 'longitude': -73.95, 'zoom': 10},…

## 5. Visualization 3: Rebalancing Flow Lines (Per Agent)

Show vehicle rebalancing flows as lines between regions, with thickness representing flow volume.

In [9]:
def create_rebalancing_flow_data(agent_rebFlow, zones_gdf, timestep=None, min_flow=1):
    """
    Create flow lines for rebalancing visualization.
    If timestep is None, sum across all timesteps.
    min_flow: minimum flow to display (filter out small flows)
    """
    flow_data = []
    
    # Get centroids for each zone
    centroids = zones_gdf.geometry.centroid
    
    for agent_id in [0, 1]:
        for (i, j), time_flows in agent_rebFlow[agent_id].items():
            if i >= len(zones_gdf) or j >= len(zones_gdf):
                continue
            
            # Calculate flow volume
            if timestep is not None:
                if timestep in time_flows:
                    flow = time_flows[timestep]
                else:
                    continue
            else:
                flow = sum(time_flows.values())
            
            if flow < min_flow:
                continue
            
            # Create line geometry from origin to destination centroid
            origin = centroids.iloc[i]
            destination = centroids.iloc[j]
            line = LineString([origin, destination])
            
            flow_data.append({
                'origin_id': i,
                'dest_id': j,
                'agent_id': agent_id,
                'agent_name': f'Agent {agent_id}',
                'flow_volume': flow,
                'origin_lon': origin.x,
                'origin_lat': origin.y,
                'dest_lon': destination.x,
                'dest_lat': destination.y,
                'geometry': line
            })
    
    return gpd.GeoDataFrame(flow_data, crs=zones_gdf.crs)

# Create rebalancing flow data
flow_gdf = create_rebalancing_flow_data(agent_rebFlow, zones_gdf, min_flow=2)
print(f"Created {len(flow_gdf)} rebalancing flow lines")
print(f"Total flow volume: {flow_gdf['flow_volume'].sum():.0f} vehicles")
flow_gdf.head()

Created 282 rebalancing flow lines
Total flow volume: 4611 vehicles


/tmp/ipykernel_2592361/578659475.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = zones_gdf.geometry.centroid


,origin_id,dest_id,agent_id,agent_name,flow_volume,origin_lon,origin_lat,dest_lon,dest_lat,geometry
0,0,0,0,Agent 0,21,-73.934977,40.794908,-73.934977,40.794908,"LINESTRING (-73.93498 40.79491, -73.93498 40.7..."
1,0,1,0,Agent 0,20,-73.934977,40.794908,-74.011311,40.707636,"LINESTRING (-73.93498 40.79491, -74.01131 40.7..."
2,0,2,0,Agent 0,5,-73.934977,40.794908,-73.944734,40.813689,"LINESTRING (-73.93498 40.79491, -73.94473 40.8..."
3,0,3,0,Agent 0,5,-73.934977,40.794908,-74.001559,40.729836,"LINESTRING (-73.93498 40.79491, -74.00156 40.7..."
4,0,4,0,Agent 0,16,-73.934977,40.794908,-73.985543,40.719596,"LINESTRING (-73.93498 40.79491, -73.98554 40.7..."


In [10]:
# Visualize rebalancing flows with Kepler.gl
map_flows = KeplerGl(height=600)
map_flows.add_data(data=flow_gdf, name='Rebalancing Flows')

flow_config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': 40.75,
            'longitude': -73.95,
            'zoom': 10
        },
        'visState': {
            'filters': [
                {
                    'dataId': ['Rebalancing Flows'],
                    'id': 'agent_filter',
                    'name': ['agent_name'],
                    'type': 'multiSelect',
                    'value': ['Agent 0', 'Agent 1'],
                    'enlarged': True
                }
            ],
            'layers': [
                {
                    'type': 'arc',
                    'config': {
                        'dataId': 'Rebalancing Flows',
                        'label': 'Rebalancing Arcs',
                        'color': [18, 147, 154],
                        'columns': {
                            'lat0': 'origin_lat',
                            'lng0': 'origin_lon',
                            'lat1': 'dest_lat',
                            'lng1': 'dest_lon'
                        },
                        'isVisible': True,
                        'visConfig': {
                            'opacity': 0.8,
                            'thickness': 2,
                            'colorRange': {
                                'name': 'Global Warming',
                                'type': 'sequential',
                                'category': 'Uber',
                                'colors': ['#5A1846', '#900C3F', '#C70039', '#E3611C', '#F1920E', '#FFC300']
                            },
                            'sizeRange': [0, 10],
                            'targetColor': None
                        },
                        'colorField': {'name': 'flow_volume', 'type': 'integer'},
                        'colorScale': 'quantile',
                        'sizeField': {'name': 'flow_volume', 'type': 'integer'},
                        'sizeScale': 'sqrt'
                    }
                }
            ]
        }
    }
}

map_flows.config = flow_config
map_flows

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'mapState': {'latitude': 40.75, 'longitude': -73.95, 'zoom': 10},…

## 6. Visualization 4: Net Rebalancing Flow Difference

Show the difference in rebalancing patterns between agents. Helps identify which agent is more active in which corridors.

In [11]:
def create_net_flow_difference_data(agent_rebFlow, zones_gdf, min_flow_diff=2):
    """
    Calculate net flow differences (Agent 0 - Agent 1) for each O-D pair.
    Positive means Agent 0 has more flow, negative means Agent 1 has more.
    """
    flow_diff_data = []
    centroids = zones_gdf.geometry.centroid
    
    # Get all O-D pairs from both agents
    all_pairs = set()
    for agent_id in [0, 1]:
        all_pairs.update(agent_rebFlow[agent_id].keys())
    
    for (i, j) in all_pairs:
        if i >= len(zones_gdf) or j >= len(zones_gdf):
            continue
        
        # Sum flows across all timesteps for each agent
        flow_0 = sum(agent_rebFlow[0].get((i, j), {}).values())
        flow_1 = sum(agent_rebFlow[1].get((i, j), {}).values())
        
        flow_diff = flow_0 - flow_1
        
        if abs(flow_diff) < min_flow_diff:
            continue
        
        origin = centroids.iloc[i]
        destination = centroids.iloc[j]
        line = LineString([origin, destination])
        
        flow_diff_data.append({
            'origin_id': i,
            'dest_id': j,
            'flow_agent0': flow_0,
            'flow_agent1': flow_1,
            'flow_diff': flow_diff,
            'dominant_agent': f'Agent {0 if flow_diff > 0 else 1}',
            'origin_lon': origin.x,
            'origin_lat': origin.y,
            'dest_lon': destination.x,
            'dest_lat': destination.y,
            'geometry': line
        })
    
    return gpd.GeoDataFrame(flow_diff_data, crs=zones_gdf.crs)

net_flow_gdf = create_net_flow_difference_data(agent_rebFlow, zones_gdf)
print(f"Net flow differences: {len(net_flow_gdf)} corridors")
print(f"Flow diff range: {net_flow_gdf['flow_diff'].min():.0f} to {net_flow_gdf['flow_diff'].max():.0f}")
net_flow_gdf.head()

Net flow differences: 123 corridors
Flow diff range: -36 to 31


/tmp/ipykernel_2592361/1032630043.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = zones_gdf.geometry.centroid


,origin_id,dest_id,flow_agent0,flow_agent1,flow_diff,dominant_agent,origin_lon,origin_lat,dest_lon,dest_lat,geometry
0,4,0,11,26,-15,Agent 1,-73.985543,40.719596,-73.934977,40.794908,"LINESTRING (-73.98554 40.7196, -73.93498 40.79..."
1,4,9,27,15,12,Agent 0,-73.985543,40.719596,-73.955030,40.818050,"LINESTRING (-73.98554 40.7196, -73.95503 40.81..."
2,5,1,23,4,19,Agent 0,-73.955767,40.771543,-74.011311,40.707636,"LINESTRING (-73.95577 40.77154, -74.01131 40.7..."
3,8,0,7,18,-11,Agent 1,-73.997526,40.755432,-73.934977,40.794908,"LINESTRING (-73.99753 40.75543, -73.93498 40.7..."
4,5,10,21,17,4,Agent 0,-73.955767,40.771543,-73.931518,40.855116,"LINESTRING (-73.95577 40.77154, -73.93152 40.8..."


In [12]:
# Visualize net flow differences
map_net_flows = KeplerGl(height=600)
map_net_flows.add_data(data=net_flow_gdf, name='Net Flow Difference')

net_flow_config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': 40.75,
            'longitude': -73.95,
            'zoom': 10
        },
        'visState': {
            'layers': [
                {
                    'type': 'arc',
                    'config': {
                        'dataId': 'Net Flow Difference',
                        'label': 'Net Rebalancing Difference',
                        'color': [18, 147, 154],
                        'columns': {
                            'lat0': 'origin_lat',
                            'lng0': 'origin_lon',
                            'lat1': 'dest_lat',
                            'lng1': 'dest_lon'
                        },
                        'isVisible': True,
                        'visConfig': {
                            'opacity': 0.8,
                            'thickness': 2,
                            'colorRange': {
                                'name': 'Diverging RdYlBu',
                                'type': 'diverging',
                                'category': 'ColorBrewer',
                                'colors': ['#D73027', '#F46D43', '#FDAE61', '#FEE090', '#E0F3F8', '#ABD9E9', '#74ADD1', '#4575B4']
                            },
                            'sizeRange': [0, 10],
                            'targetColor': None
                        },
                        'colorField': {'name': 'flow_diff', 'type': 'real'},
                        'colorScale': 'quantize',
                        'sizeField': {'name': 'flow_diff', 'type': 'real'},
                        'sizeScale': 'sqrt'
                    }
                }
            ]
        }
    }
}

map_net_flows.config = net_flow_config
map_net_flows

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'mapState': {'latitude': 40.75, 'longitude': -73.95, 'zoom': 10},…

## 7. Visualization 5: Vehicle Availability Heatmap with 3D Elevation

Show vehicle distribution across regions using 3D elevation for dramatic effect.

In [13]:
def create_vehicle_availability_data(agent_acc, zones_gdf, timestep=None):
    """
    Create vehicle availability data for each region.
    If timestep is None, use average across all timesteps.
    """
    availability_data = []
    
    for agent_id in [0, 1]:
        for region_id, time_acc in agent_acc[agent_id].items():
            if region_id >= len(zones_gdf):
                continue
            
            if timestep is not None:
                if timestep in time_acc:
                    vehicles = time_acc[timestep]
                else:
                    continue
            else:
                vehicles = np.mean(list(time_acc.values()))
            
            availability_data.append({
                'region_id': region_id,
                'agent_id': agent_id,
                'agent_name': f'Agent {agent_id}',
                'vehicles': vehicles,
                'geometry': zones_gdf.iloc[region_id].geometry
            })
    
    return gpd.GeoDataFrame(availability_data, crs=zones_gdf.crs)

availability_gdf = create_vehicle_availability_data(agent_acc, zones_gdf)
print(f"Vehicle availability data: {len(availability_gdf)} records")
availability_gdf.head()

Vehicle availability data: 24 records


,region_id,agent_id,agent_name,vehicles,geometry
0,0,0,Agent 0,22.95,"MULTIPOLYGON (((-73.93381 40.81651, -73.93383 ..."
1,1,0,Agent 0,22.15,"MULTIPOLYGON (((-74.01114 40.72581, -74.01102 ..."
2,2,0,Agent 0,22.65,"MULTIPOLYGON (((-73.93445 40.83598, -73.93456 ..."
3,3,0,Agent 0,20.55,"MULTIPOLYGON (((-73.99684 40.73736, -73.99362 ..."
4,4,0,Agent 0,18.50,"MULTIPOLYGON (((-73.98878 40.73397, -73.98718 ..."


In [14]:
# 3D visualization of vehicle availability
map_3d = KeplerGl(height=600)
map_3d.add_data(data=availability_gdf, name='Vehicle Availability')

availability_config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': 40.75,
            'longitude': -73.95,
            'zoom': 10,
            'pitch': 50,
            'bearing': 0
        },
        'visState': {
            'filters': [
                {
                    'dataId': ['Vehicle Availability'],
                    'id': 'agent_filter',
                    'name': ['agent_name'],
                    'type': 'multiSelect',
                    'value': ['Agent 0', 'Agent 1'],
                    'enlarged': True
                }
            ],
            'layers': [
                {
                    'type': 'geojson',
                    'config': {
                        'dataId': 'Vehicle Availability',
                        'label': '3D Vehicle Distribution',
                        'color': [18, 147, 154],
                        'columns': {'geojson': 'geometry'},
                        'isVisible': True,
                        'visConfig': {
                            'opacity': 0.8,
                            'strokeOpacity': 0.8,
                            'thickness': 0.5,
                            'strokeColor': [255, 255, 255],
                            'colorRange': {
                                'name': 'Uber Viz Sequential 2',
                                'type': 'sequential',
                                'category': 'Uber',
                                'colors': ['#E6FAFA', '#C1E5E6', '#9DD0D4', '#75BBC1', '#4BA7AF', '#00939C']
                            },
                            'filled': True,
                            'stroked': True,
                            'enable3d': True,
                            'elevationScale': 10,
                            'wireframe': False
                        },
                        'colorField': {'name': 'vehicles', 'type': 'real'},
                        'colorScale': 'quantile',
                        'heightField': {'name': 'vehicles', 'type': 'real'},
                        'heightScale': 'linear'
                    }
                }
            ]
        }
    }
}

map_3d.config = availability_config
map_3d

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'mapState': {'latitude': 40.75, 'longitude': -73.95, 'zoom': 10, …

## 8. Visualization 6: Temporal Animation

Create data with timestamps to enable Kepler.gl's time-series animation feature.

In [15]:
def create_temporal_flow_data(agent_rebFlow, zones_gdf, base_datetime=None):
    """
    Create time-stamped flow data for animation.
    Each timestep gets an actual datetime for Kepler.gl animation.
    """
    if base_datetime is None:
        base_datetime = datetime(2024, 1, 1, 8, 0, 0)  # Start at 8 AM
    
    temporal_flow_data = []
    centroids = zones_gdf.geometry.centroid
    
    for agent_id in [0, 1]:
        for (i, j), time_flows in agent_rebFlow[agent_id].items():
            if i >= len(zones_gdf) or j >= len(zones_gdf):
                continue
            
            for timestep, flow in time_flows.items():
                if flow < 1:
                    continue
                
                # Calculate datetime for this timestep (assuming 3 min intervals)
                dt = base_datetime + timedelta(minutes=3 * timestep)
                
                origin = centroids.iloc[i]
                destination = centroids.iloc[j]
                line = LineString([origin, destination])
                
                temporal_flow_data.append({
                    'origin_id': i,
                    'dest_id': j,
                    'agent_id': agent_id,
                    'agent_name': f'Agent {agent_id}',
                    'timestep': timestep,
                    'datetime': dt.isoformat(),
                    'flow_volume': flow,
                    'origin_lon': origin.x,
                    'origin_lat': origin.y,
                    'dest_lon': destination.x,
                    'dest_lat': destination.y,
                    'geometry': line
                })
    
    return gpd.GeoDataFrame(temporal_flow_data, crs=zones_gdf.crs)

temporal_flow_gdf = create_temporal_flow_data(agent_rebFlow, zones_gdf)
print(f"Temporal flow data: {len(temporal_flow_gdf)} flow records across time")
print(f"Time range: {temporal_flow_gdf['datetime'].min()} to {temporal_flow_gdf['datetime'].max()}")
temporal_flow_gdf.head()

Temporal flow data: 923 flow records across time
Time range: 2024-01-01T08:00:00 to 2024-01-01T08:57:00


/tmp/ipykernel_2592361/3891481663.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = zones_gdf.geometry.centroid


,origin_id,dest_id,agent_id,agent_name,timestep,datetime,flow_volume,origin_lon,origin_lat,dest_lon,dest_lat,geometry
0,0,0,0,Agent 0,2,2024-01-01T08:06:00,9,-73.934977,40.794908,-73.934977,40.794908,"LINESTRING (-73.93498 40.79491, -73.93498 40.7..."
1,0,0,0,Agent 0,11,2024-01-01T08:33:00,5,-73.934977,40.794908,-73.934977,40.794908,"LINESTRING (-73.93498 40.79491, -73.93498 40.7..."
2,0,0,0,Agent 0,12,2024-01-01T08:36:00,3,-73.934977,40.794908,-73.934977,40.794908,"LINESTRING (-73.93498 40.79491, -73.93498 40.7..."
3,0,0,0,Agent 0,14,2024-01-01T08:42:00,4,-73.934977,40.794908,-73.934977,40.794908,"LINESTRING (-73.93498 40.79491, -73.93498 40.7..."
4,0,1,0,Agent 0,4,2024-01-01T08:12:00,9,-73.934977,40.794908,-74.011311,40.707636,"LINESTRING (-73.93498 40.79491, -74.01131 40.7..."


In [16]:
# Animated flow visualization
map_temporal = KeplerGl(height=600)
map_temporal.add_data(data=temporal_flow_gdf, name='Temporal Flows')

temporal_config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': 40.75,
            'longitude': -73.95,
            'zoom': 10
        },
        'visState': {
            'filters': [
                {
                    'dataId': ['Temporal Flows'],
                    'id': 'time_filter',
                    'name': ['datetime'],
                    'type': 'timeRange',
                    'enlarged': True,
                    'animationWindow': 'free'
                },
                {
                    'dataId': ['Temporal Flows'],
                    'id': 'agent_filter',
                    'name': ['agent_name'],
                    'type': 'multiSelect',
                    'value': ['Agent 0', 'Agent 1'],
                    'enlarged': True
                }
            ],
            'layers': [
                {
                    'type': 'arc',
                    'config': {
                        'dataId': 'Temporal Flows',
                        'label': 'Animated Rebalancing',
                        'color': [18, 147, 154],
                        'columns': {
                            'lat0': 'origin_lat',
                            'lng0': 'origin_lon',
                            'lat1': 'dest_lat',
                            'lng1': 'dest_lon'
                        },
                        'isVisible': True,
                        'visConfig': {
                            'opacity': 0.8,
                            'thickness': 2,
                            'colorRange': {
                                'name': 'Global Warming',
                                'type': 'sequential',
                                'category': 'Uber',
                                'colors': ['#5A1846', '#900C3F', '#C70039', '#E3611C', '#F1920E', '#FFC300']
                            },
                            'sizeRange': [0, 10],
                            'targetColor': None
                        },
                        'colorField': {'name': 'agent_id', 'type': 'integer'},
                        'colorScale': 'ordinal',
                        'sizeField': {'name': 'flow_volume', 'type': 'integer'},
                        'sizeScale': 'sqrt'
                    }
                }
            ]
        }
    }
}

map_temporal.config = temporal_config
map_temporal

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'mapState': {'latitude': 40.75, 'longitude': -73.95, 'zoom': 10},…

## 9. Combined Dashboard View

Combine multiple layers for a comprehensive view.

In [17]:
# Create combined visualization
map_combined = KeplerGl(height=700)
map_combined.add_data(data=zones_gdf, name='NYC Zones')
map_combined.add_data(data=price_gdf, name='Pricing')
map_combined.add_data(data=flow_gdf, name='Flows')
map_combined.add_data(data=availability_gdf, name='Vehicles')

# Users can toggle layers on/off in the UI
map_combined

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'NYC Zones': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 'columns': ['districtcode', 'sha…

## 10. Export Configurations and Data

Save your maps and data for later use or sharing.

In [ ]:
# Save map configuration (can be reloaded later)
# map_combined.save_to_html(file_name='combined_visualization.html')

# Save processed data for reuse
# price_gdf.to_file('processed_data/price_heatmap.geojson', driver='GeoJSON')
# flow_gdf.to_file('processed_data/rebalancing_flows.geojson', driver='GeoJSON')
# temporal_flow_gdf.to_file('processed_data/temporal_flows.geojson', driver='GeoJSON')

print("To save visualizations, uncomment the save commands above")

## 11. Loading Real Model Data

When you have trained model results, use this template to load and visualize them.

In [ ]:
# Template for loading real data from your model

def load_model_results(checkpoint_path, test_episode_data_path):
    """
    Load pricing and rebalancing data from your trained model.
    
    Expected data format after running test episodes:
    - agent_price: dict[agent_id][(i,j)][t] = price_scalar
    - agent_rebFlow: dict[agent_id][(i,j)][t] = num_vehicles
    - agent_acc: dict[agent_id][region][t] = num_available_vehicles
    - agent_paxFlow: dict[agent_id][(i,j)][t] = num_passenger_vehicles
    
    Returns:
        dict with keys: 'agent_price', 'agent_rebFlow', 'agent_acc', 'agent_paxFlow'
    """
    # TODO: Implement loading from your model's output format
    # This will depend on how you save test results in main_a2c_multi_agent.py
    
    # Example structure:
    # import pickle
    # with open(test_episode_data_path, 'rb') as f:
    #     data = pickle.load(f)
    # 
    # return {
    #     'agent_price': data['agent_price'],
    #     'agent_rebFlow': data['agent_rebFlow'],
    #     'agent_acc': data['agent_acc'],
    #     'agent_paxFlow': data.get('agent_paxFlow', {})
    # }
    
    pass

# Usage:
# real_data = load_model_results(
#     checkpoint_path='ckpt/base_case_manhattan_mode2_30k_dual_agent_od_down_scaled_agent1_test.pth',
#     test_episode_data_path='saved_files/test_results/episode_data.pkl'
# )
# 
# # Then use the same visualization functions as above:
# real_price_gdf = create_price_heatmap_data(real_data['agent_price'], zones_gdf)
# real_flow_gdf = create_rebalancing_flow_data(real_data['agent_rebFlow'], zones_gdf)

## Summary of Visualization Types

### **For Pricing:**
1. **Regional Price Heatmap** - Shows average price scalar by region for each agent
2. **Price Difference Heatmap** - Shows competitive dynamics (which agent prices lower where)

### **For Rebalancing:**
3. **Flow Arc Diagram** - Shows vehicle movements as directional arcs (per agent)
4. **Net Flow Difference** - Shows which agent dominates each corridor
5. **Temporal Animation** - Shows how flows evolve over time

### **For Vehicle Distribution:**
6. **3D Availability Map** - Shows vehicle concentration using elevation

### **Combined:**
7. **Multi-layer Dashboard** - All visualizations together with toggleable layers

### **Key Features:**
- Filter by agent (toggle Agent 0 vs Agent 1)
- Time slider for temporal data
- Color coding by metric (price, flow volume, etc.)
- Arc thickness represents flow volume
- Diverging colors show competition/differences
- Interactive tooltips with detailed information